In [ ]:
import os

from ultralytics import YOLO

import numpy as np
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/asrulsibaoel/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
DATASET_PATH = "data/offroad-dataset-ii-instance"
IMAGE_TEST_PATH = "data/test_samples/sample1.jpg"

In [ ]:
# Load Pretrained YOLOv8-seg Model
model = YOLO("yolov8-seg.pt")

In [ ]:
# Training Parameters
epochs = 50
image_size = 640
batch = 16

In [ ]:


# Dataset YAML
# Create a dataset YAML file pointing to your drone/satellite images and annotations.

# Train the Model
model.train(
    data=os.path.join(DATASET_PATH, "data.yaml"),  # Path to your dataset YAML
    epochs=epochs,
    imgsz=image_size,
    batch=batch,
    device=0  # Use GPU if available
)

# Evaluate the Model
results = model.val()

results

In [ ]:
image_path = IMAGE_TEST_PATH

## Inference Example

In [ ]:
image = cv2.imread(image_path)
results = model.predict(source=image, save=True, conf=0.5)

# Visualize Results
for result in results:
    mask = result.masks.data[0].numpy()  # Access first mask
    cv2.imshow("Segmented Road", mask)
    cv2.waitKey(0)

## Road Size Calculation  

Approach:
1. Segment the Road  
Use the segmentation output to identify the road pixels or regions.

2. Calculate Dimensions  
Convert pixel dimensions to real-world measurements using the image's spatial resolution or ground sampling distance (GSD).

3. Estimate Width and Area  
Extract road contours and calculate the width, length, and area.

4. Real-World Scaling  
Use metadata such as drone altitude, camera resolution, or satellite GSD for accurate scaling.

### Extract Road Masks  
After running the segmentation model, extract the mask of the road:

In [ ]:
masks = results[0].masks.data.numpy()  # Extract masks

# Combine masks into a single binary mask (if necessary)
road_mask = np.zeros_like(image[:, :, 0])
for mask in masks:
    road_mask = np.maximum(road_mask, mask)

### Find Contours and Calculate Size  
Using OpenCV, find the road's contours and calculate its dimensions:

In [ ]:
# Find contours
contours, _ = cv2.findContours(road_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Analyze contours
for contour in contours:
    # Approximate the contour to a polygon
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    # Get bounding box
    x, y, w, h = cv2.boundingRect(approx)
    road_width = w  # Width in pixels
    road_length = h  # Length in pixels
    
    # Calculate area
    road_area = cv2.contourArea(contour)
    
    print(f"Road Width (pixels): {road_width}")
    print(f"Road Length (pixels): {road_length}")
    print(f"Road Area (pixels): {road_area}")


### Convert Pixels to Real-World Units  
If using drone or satellite imagery:
- **Drone Images**:  
$\ RealWorldWidth(m) = {\frac{Width(px) \times SensorWidth(mm)}{ImageWidth(px)}}\times{\frac{Altitude(m)}{FocalLength(m)}}$

- **Satelite Images**  
Use Ground Sampling Distance (GSD) to scale pixel dimensions:  
$\ RealWorldWidth(m) = Width(px) \times GSD(m/pixel)$

In [ ]:
# Example for satellite/drone image scaling
pixel_width = road_width
pixel_length = road_length
gsd = 0.1  # Example: 0.1 meters per pixel (adjust based on metadata)

real_width = pixel_width * gsd
real_length = pixel_length * gsd

print(f"Road Width (meters): {real_width}")
print(f"Road Length (meters): {real_length}")


## Visualize The Results
Overlay measurements on the image for better interpretation:

In [2]:
output_image = image.copy()
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(output_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(output_image, f"Width: {real_width:.2f}m", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imshow("Road Dimensions", output_image)
cv2.waitKey(0)


NameError: name 'image' is not defined